In [1]:
import json
import pendulum
from airflow.sdk import dag, task

In [5]:
@dag(
    schedule=None,
    start_date=pendulum.datetime(2021, 1, 1, tz="UTC"),
    catchup=False,
    tags=['example'],
)
def tutorial_taskflow_api():
    @task()
    def extract():
        data_string = '{"1001": 301.27, "1002": 433.21, "1003": 502.22}'
        order_data_dict = json.loads(data_string)
        return order_data_dict
    @task(multiple_outputs=True)
    def transform(order_data_dict: dict):
        total_order_value = 0
        for v in order_data_dict.values():
            total_order_value += v

        return {"total_order_value": total_order_value}
    @task(retries=3)
    def load(v: float):
        print(f"Total order value is: {v:.2f}")

    order_data = extract()
    order_summary = transform(order_data)
    load(order_summary["total_order_value"])

tutorial_taskflow_api()

<DAG: tutorial_taskflow_api>